In [ ]:
from pyNN.utility import plotting

#try:
#    with open('neuronal_data.p','wb') as f:
#        data = pickle.load(f)
#except:
if False:
    pass
else:
    from neuronunit.tests import dynamics

    import socket
    from math import *
    from pyNN.utility import get_simulator, Timer, ProgressBar, init_logging, normalized_filename
    from pyNN.random import NumpyRNG, RandomDistribution
    import pyNN
    import pyNN.neuron as sim
    #import pyNN.spiNNaker as sim
    import socket


    # === Configure the simulator ================================================
    #sim = pyNN.neuron

    use_csa = False
    debug = True
    #if use_csa:
    #    import csa

    if debug:
        init_logging(None, debug=True)

    timer = Timer()

    # === Define parameters ========================================================

    threads  = 1
    rngseed  = 98765
    parallel_safe = True

    n        = 4000  # number of cells
    r_ei     = 4.0   # number of excitatory cells:number of inhibitory cells
    pconn    = 0.02  # connection probability
    stim_dur = 50.   # (ms) duration of random stimulation
    rate     = 100.  # (Hz) frequency of the random stimulation

    dt       = 0.1   # (ms) simulation timestep
    tstop    = 3490  # (ms) simulaton duration
    delay    = 0.2

    # Cell parameters
    area     = 20000. # (µm²)
    tau_m    = 20.    # (ms)
    cm       = 1.     # (µF/cm²)
    g_leak   = 5e-5   # (S/cm²)
    #benchmark = "CUBA"
    benchmark = "COBA"

    if benchmark == "COBA":
        E_leak   = -60.  # (mV)
    elif benchmark == "CUBA":
        E_leak   = -49.  # (mV)
    v_thresh = -50.   # (mV)
    v_reset  = -60.   # (mV)
    t_refrac = 5.     # (ms) (clamped at v_reset)
    v_mean   = -60.   # (mV) 'mean' membrane potential, for calculating CUBA weights
    tau_exc  = 5.     # (ms)
    tau_inh  = 10.    # (ms)

    # Synapse parameters
    if benchmark == "COBA":
        Gexc = 4.     # (nS)
        Ginh = 51.    # (nS)
    elif benchmark == "CUBA":
        Gexc = 0.27   # (nS) #Those weights should be similar to the COBA weights
        Ginh = 4.5    # (nS) # but the delpolarising drift should be taken into account
    Erev_exc = 0.     # (mV)
    Erev_inh = -80.   # (mV)

    ### what is the synaptic delay???

    # === Calculate derived parameters =============================================

    area  = area*1e-8                     # convert to cm²
    cm    = cm*area*1000                  # convert to nF
    Rm    = 1e-6/(g_leak*area)            # membrane resistance in MΩ
    assert tau_m == cm*Rm                 # just to check
    n_exc = int(round((n*r_ei/(1+r_ei)))) # number of excitatory cells
    n_inh = n - n_exc                     # number of inhibitory cells
    if benchmark == "COBA":
        celltype = sim.IF_cond_exp
        w_exc    = Gexc*2.4e-10             # We convert conductances to uS
        w_inh    = Ginh*4e-10
    elif benchmark == "CUBA":
        celltype = sim.IF_curr_exp
        w_exc = 2.4e-10*Gexc*(Erev_exc - v_mean)  # (nA) weight of excitatory synapses
        w_inh = 4e-10*Ginh*(Erev_inh - v_mean)  # (nA)
        assert w_exc > 0; assert w_inh < 0

    # === Build the network ========================================================

    extra = {'threads' : threads,
             'filename': "va_%s.xml" % benchmark,
             'label': 'VA'}
    #if simulator == "neuroml":
    #    extra["file"] = "VAbenchmarks.xml"

    node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=1.0, **extra)
    np = sim.num_processes()

    host_name = socket.gethostname()
    print("Host #%d is on %s" % (node_id + 1, host_name))

    print("%s Initialising the simulator with %d thread(s)..." % (node_id, extra['threads']))

    cell_params = {
        'tau_m'      : tau_m,    'tau_syn_E'  : tau_exc,  'tau_syn_I'  : tau_inh,
        'v_rest'     : E_leak,   'v_reset'    : v_reset,  'v_thresh'   : v_thresh,
        'cm'         : cm,       'tau_refrac' : t_refrac}

    if (benchmark == "COBA"):
        cell_params['e_rev_E'] = Erev_exc
        cell_params['e_rev_I'] = Erev_inh

    timer.start()

    print("%s Creating cell populations..." % node_id)
    use_views = True
    if use_views:
        # create a single population of neurons, and then use population views to define
        # excitatory and inhibitory sub-populations
        all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
        exc_cells = all_cells[:n_exc]
        exc_cells.label = "Excitatory cells"
        inh_cells = all_cells[n_exc:]
        inh_cells.label = "Inhibitory cells"
    else:
        # create separate populations for excitatory and inhibitory neurons
        exc_cells = sim.Population(n_exc, celltype(**cell_params), label="Excitatory_Cells")
        inh_cells = sim.Population(n_inh, celltype(**cell_params), label="Inhibitory_Cells")
        if use_assembly:
            # group the populations into an assembly
            all_cells = exc_cells + inh_cells

    if benchmark == "COBA":
        ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), label="expoisson")
        rconn = 0.01
        ext_conn = sim.FixedProbabilityConnector(rconn)
        ext_syn = sim.StaticSynapse(weight=0.1)

    print("%s Initialising membrane potential to random values..." % node_id)
    rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
    uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
    all_cells.record(['v','spikes'])  # , 'u'])

    if use_views:
        all_cells.initialize(v=uniformDistr)
    else:
        exc_cells.initialize(v=uniformDistr)-po
        inh_cells.initialize(v=uniformDistr)

    print("%s Connecting populations..." % node_id)
    progress_bar = ProgressBar(width=20)
    if use_csa:
        connector = sim.CSAConnector(csa.cset(csa.random(pconn)))
    else:
        connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
    exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
    inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

    import pickle
    with open('unit_test/connections.p','rb') as f:
       [ post_inh,pre_inh,pre_exc,post_exc] = pickle.load(f)
    pre_inh = inh_cells[pre_inh]
    post_inh = all_cells[post_inh]
    pre_exc = exc_cells[pre_exc]
    post_exc = all_cells[post_exc]

    connections = {}

    #connections = {}
    #connections['exc'] = sim.Projection(pre_inh, post_inh, connector, exc_syn, receptor_type='excitatory')
    #connections['inh'] = sim.Projection(pre_exc, post_exc, connector, inh_syn, receptor_type='inhibitory')
    
    if (benchmark == "COBA"):
        connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

        connections['e2e'] = sim.Projection(exc_cells, exc_cells, connector, exc_syn, receptor_type='excitatory')
        connections['e2i'] = sim.Projection(exc_cells, inh_cells, connector, exc_syn, receptor_type='excitatory')
        connections['i2e'] = sim.Projection(inh_cells, exc_cells, connector, inh_syn, receptor_type='inhibitory')
        connections['i2i'] = sim.Projection(inh_cells, inh_cells, connector, inh_syn, receptor_type='inhibitory')



    if (benchmark == "COBA"):
        connections['ext2e'] = sim.Projection(ext_stim, exc_cells, ext_conn, ext_syn, receptor_type='excitatory')
        connections['ext2i'] = sim.Projection(ext_stim, inh_cells, ext_conn, ext_syn, receptor_type='excitatory')

    if use_views or use_assembly:
        connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
        connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
        if (benchmark == "COBA"):
            connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')
    else:
        connections['e2e'] = sim.Projection(exc_cells, exc_cells, connector, exc_syn, receptor_type='excitatory')
        connections['e2i'] = sim.Projection(exc_cells, inh_cells, connector, exc_syn, receptor_type='excitatory')
        connections['i2e'] = sim.Projection(inh_cells, exc_cells, connector, inh_syn, receptor_type='inhibitory')
        connections['i2i'] = sim.Projection(inh_cells, inh_cells, connector, inh_syn, receptor_type='inhibitory')
        if (benchmark == "COBA"):
            connections['ext2e'] = sim.Projection(ext_stim, exc_cells, ext_conn, ext_syn, receptor_type='excitatory')
            connections['ext2i'] = sim.Projection(ext_stim, inh_cells, ext_conn, ext_syn, receptor_type='excitatory')
    
    # === Setup recording ==========================================================
    print("%s Setting up recording..." % node_id)
    if use_views or use_assembly:
        all_cells.record('spikes')
        exc_cells[[0, 1]].record('v')
    else:
        exc_cells.record('spikes')
        inh_cells.record('spikes')
        exc_cells[0, 1].record('v')

    buildCPUTime = timer.diff()

    # === Save connections to file =================================================

    #for prj in connections.keys():
        #connections[prj].saveConnections('Results/VAbenchmark_%s_%s_%s_np%d.conn' % (benchmark, prj, options.simulator, np))
    saveCPUTime = timer.diff()

    # === Run simulation ===========================================================

    print("%d Running simulation..." % node_id)

    sim.run(tstop)

    simCPUTime = timer.diff()

    E_count = exc_cells.mean_spike_count()
    I_count = inh_cells.mean_spike_count()

    # === Print results to file ====================================================

    print("%d Writing data to file..." % node_id)

    #filename = normalized_filename("Results", "VAbenchmarks_%s_exc" % options.benchmark, "pkl",
    #                               options.simulator, np)
    #exc_cells.write_data(filename,
    #                     annotations={'script_name': __file__})
    #inh_cells.write_data(filename.replace("exc", "inh"),
    #                     annotations={'script_name': __file__})

    writeCPUTime = timer.diff()

    if use_views or use_assembly:
        connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
                                              connections['inh'].size())
    else:
        connections = u"%d e→e  %d e→i  %d i→e  %d i→i" % (connections['e2e'].size(),
                                                           connections['e2i'].size(),
                                                           connections['i2e'].size(),
                                                           connections['i2i'].size())

    if node_id == 0:
        print("\n--- Vogels-Abbott Network Simulation ---")
        print("Nodes                  : %d" % np)
        #print("Simulation type        : %s" % options.benchmark)
        print("Number of Neurons      : %d" % n)
        print("Number of Synapses     : %s" % connections)
        print("Excitatory conductance : %g nS" % Gexc)
        print("Inhibitory conductance : %g nS" % Ginh)
        print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / tstop,))
        print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / tstop,))
        print("Build time             : %g s" % buildCPUTime)
        #print("Save connections time  : %g s" % saveCPUTime)
        print("Simulation time        : %g s" % simCPUTime)
        print("Writing time           : %g s" % writeCPUTime)


    # === Finished with simulator ==================================================

    from pyNN.utility.plotting import Figure, Panel
    #figure_filename = filename.replace("pkl", "png")
    data = all_cells.get_data().segments[0]
    v = data.filter(name="v")[0]
    #u = data.filter(name="u")[0]
    Figure(
        Panel(v, ylabel="Membrane potential (mV)", xticks=True,
              xlabel="Time (ms)", yticks=True),
        #Panel(u, ylabel="u variable (units?)"),
        annotations="Simulated with" 
    )

    # === Clean up and quit ========================================================
    #spikes = all_cells.get_data("spikes").segments[0]
    sim.end()
    spikes_in = all_cells.get_data()
    from pyNN.utility.plotting import Figure, Panel, comparison_plot, plot_spiketrains


    # === Clean up and quit ========================================================
    data = all_cells.get_data().segments[0]



    #figure_filename = filename.replace("pkl", "png")
    #data = all_cells.get_data().segments[0]
    v = data.filter(name="v")[0]
    #u = data.filter(name="u")[0]
    Figure(
        Panel(v, ylabel="Membrane potential (mV)", xticks=True,
              xlabel="Time (ms)", yticks=True),
        #Panel(u, ylabel="u variable (units?)"),
        annotations="Simulated with" 
    )
    from pyNN.utility import plotting
    # Plot spike trains
    import matplotlib.pyplot as plt
    import numpy as np
    def plot_spiketrains(segment):
      """
      Plots the spikes of all the cells in the given segments
      """
      for spiketrain in segment.spiketrains:
          y = np.ones_like(spiketrain) * spiketrain.annotations['source_id']
          plt.plot(spiketrain, y, '.b')
          #plt.ylabel('Neuron number')
          plt.xlabel('Spikes')





            

2018-02-23 00:58:34,279 INFO     [PyNN] Initializing membrane potential of 0 cells and 0 Populations. (/opt/conda/lib/python3.5/site-packages/pyNN/neuron/simulator.py[140]:_initialize)


Host #1 is on 287510e4b78a
0 Initialising the simulator with 1 thread(s)...
0 Creating cell populations...


2018-02-23 00:58:34,835 DEBUG    [PyNN] In Population 'All Cells', initialising v to -65.0 (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[402]:initialize)
2018-02-23 00:58:34,840 DEBUG    [PyNN] In Population 'All Cells', initialising gsyn_exc to 0.0 (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[402]:initialize)
2018-02-23 00:58:34,845 DEBUG    [PyNN] In Population 'All Cells', initialising gsyn_inh to 0.0 (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[402]:initialize)
2018-02-23 00:58:34,854 DEBUG    [PyNN] All Cells.record('['v', 'spikes']') (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[441]:record)
2018-02-23 00:58:34,854 DEBUG    [PyNN] Recorder.record(<4000 cells>) (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[222]:record)


0 Initialising membrane potential to random values...


2018-02-23 00:58:35,499 DEBUG    [PyNN] In Population 'All Cells', initialising v to RandomDistribution('uniform', {'high': -50.0, 'low': -60.0}, NumpyRNG(seed=98765)) (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[402]:initialize)
2018-02-23 00:58:35,507 WARNING  [root] PopulationView can contain only once each ID, duplicated IDs are remove (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[825]:__init__)
2018-02-23 00:58:35,508 WARNING  [root] PopulationView can contain only once each ID, duplicated IDs are remove (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[825]:__init__)
2018-02-23 00:58:35,510 WARNING  [root] PopulationView can contain only once each ID, duplicated IDs are remove (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[825]:__init__)
2018-02-23 00:58:35,511 WARNING  [root] PopulationView can contain only once each ID, duplicated IDs are remove (/opt/conda/lib/python3.5/site-packages/pyNN/common/po

...

2018-02-23 00:59:17,250 DEBUG    [PyNN] Recorder.record(<4000 cells>) (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[222]:record)
2018-02-23 00:59:17,262 DEBUG    [PyNN] view of 'Excitatory cells' with size 2.record('v') (/opt/conda/lib/python3.5/site-packages/pyNN/common/populations.py[441]:record)
2018-02-23 00:59:17,262 DEBUG    [PyNN] Recorder.record(<2 cells>) (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[222]:record)
2018-02-23 00:59:17,298 INFO     [PyNN] Initializing membrane potential of 4000 cells and 1 Populations. (/opt/conda/lib/python3.5/site-packages/pyNN/neuron/simulator.py[140]:_initialize)


0 Running simulation...


2018-02-23 00:59:17,429 DEBUG    [PyNN] default_maxstep on host #0 = 10 (/opt/conda/lib/python3.5/site-packages/pyNN/neuron/simulator.py[241]:_pre_run)
2018-02-23 00:59:17,430 DEBUG    [PyNN] local_minimum_delay on host #0 = 10 (/opt/conda/lib/python3.5/site-packages/pyNN/neuron/simulator.py[242]:_pre_run)


0 Writing data to file...

--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 4000
Number of Synapses     : 255683 e→e,i  63780 i→e,i
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 1.02051 Hz
Inhibitory rate        : 1.06805 Hz
Build time             : 42.9829 s
Simulation time        : 211.236 s
Writing time           : 0.0251548 s


2018-02-23 01:02:55,239 DEBUG    [PyNN] 0 **** ids=[   0    1    2 ..., 3997 3998 3999], channels=<neo.core.channelindex.ChannelIndex object at 0x7f38fa61fef0> (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[287]:_get_current_segment)
2018-02-23 01:03:57,205 DEBUG    [PyNN] 0 **** ids=[   0    1    2 ..., 3997 3998 3999], channels=<neo.core.channelindex.ChannelIndex object at 0x7f38907b1ba8> (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[287]:_get_current_segment)
2018-02-23 01:04:04,567 DEBUG    [PyNN] 0 **** ids=[   0    1    2 ..., 3997 3998 3999], channels=<neo.core.channelindex.ChannelIndex object at 0x7f3880c724e0> (/opt/conda/lib/python3.5/site-packages/pyNN/recording/__init__.py[287]:_get_current_segment)


In [ ]:
#spikes = all_cells.get_data("spikes").segments[0]
data = all_cells.get_data().segments[0]
plot_spiketrains(data)
import pickle
#with open('neuronal_data.p','wb') as f:
#    pickle.dump(data,f)
data = all_cells.get_data().segments[0]
with open('membrane_dynamics_balanced_file.p', 'wb') as f:
    pickle.dump(data,f)

In [ ]:
data.spiketrains


In [ ]:

nyquist = int(N/2)
frequencies = frequencies[:nyquist]
magnitude = magnitude[:nyquist]
T_max = 0.01
window = np.sin(np.pi*times/T_max)**2
signal *= [window]



df = 1/T_max
F_max = 1/dt
frequencies = np.arange(0,F_max,df)
frequencies[int(N/2):] -= F_max



from scipy.signal import periodogram
frequencies, power = periodogram(signal,fs=1/dt)
def plot_periodogram(frequencies,power):
    plt.figure(figsize=(10,4))
    plt.plot(frequencies,power)
    plt.xlabel('Frequency ($Hz$)')
    plt.ylabel('Power ($V^2/Hz$)') # Note that power is now 
                                   # a normalized density
plot_periodogram(frequencies,power)


In [ ]:
#plot_signal(times,signal)
#plot_periodogram(frequencies,power)